# A noptebook to get a table with AS3D, UBERON ID, and Sex

# Import libraries

In [49]:
import requests
import pandas as pd
from io import StringIO

# Write SPARQL query and make request

In [50]:
query = '''PREFIX ccf: <http://purl.org/ccf/>
PREFIX HRA: <https://purl.humanatlas.io/collection/hra>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX UBERON: <http://purl.obolibrary.org/obo/UBERON_>
PREFIX FMA: <http://purl.org/sig/ont/fma/fma>

SELECT DISTINCT ?as_label ?uberon_id ?sex ?ref_organ
FROM HRA:
FROM ccf:
WHERE {
  ?as_3d ccf:has_reference_organ ?ref_organ ;
		rdfs:label ?as_label_long ;
		ccf:organ_owner_sex ?sex;
		ccf:representation_of ?as_id .
	
	# ?x ccf:ccf_pref_label ?organ_label ;
	#  ?ref_organ ccf:representation_of ?organIri .
	#  ?organ_iri ccf:ccf_pref_label ?organ_label .

  FILTER(!CONTAINS(STR(?ref_organ), '/united-'))
  FILTER (str(?as_3d) != str(?ref_organ))
	BIND(REPLACE(REPLACE(STR(?as_id), STR(UBERON:), 'UBERON:'), STR(FMA:), 'FMA:') as ?uberon_id)
	BIND(REPLACE(STR(?as_label_long), STR("Spatial entity of "), '') as ?as_label)
}
'''

# set endpoint
sparql_endpoint = "http://lod.humanatlas.io/sparql"

# Set up the parameters for the request
params = {
    "query": query,
}

headers = {
    "Accept": "text/csv",
    # "Content-Type": "application/x-www-form-urlencoded"
}

# Send the request to the SPARQL endpoint
response = requests.get(sparql_endpoint, params=params, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Read the CSV response into a pandas DataFrame
    csv_data = StringIO(response.text)
    df = pd.read_csv(csv_data)
    # Print the DataFrame
    print(df)
else:
    print(f"Error: {response.status_code}")
    print(response.text)

                                               as_label       uberon_id  \
0     Spatial entity of female abdominal ostium of u...       FMA:77049   
1      Spatial entity of female anterior wall of uterus      FMA:224876   
2               Spatial entity of female body of uterus  UBERON:0009853   
3                       Spatial entity of female cervix  UBERON:0000002   
4         Spatial entity of female external cervical os  UBERON:0013760   
...                                                 ...             ...   
1168         Spatial entity of male postcentral gyrus L  UBERON:0002581   
1169         Spatial entity of male postcentral gyrus R  UBERON:0002581   
1170  Spatial entity of male posterior intermediate ...  UBERON:0002644   
1171  Spatial entity of male posterior intermediate ...  UBERON:0002644   
1172  Spatial entity of male posterior parahippocamp...  UBERON:0002657   

         sex                                          ref_organ  
0     Female  https://purl.humana

# Export result to CSV

In [53]:
df.to_csv("output/as3d_uberon_sex.csv", index=False)